# News Hour - Part III

In this notebook, we will begin to set up some very basic mata functionality and begin the estimation process. Essentially, what we do is:
1. Set up the log-likelihood for our pricing model, and
2. Estimate the price model, seperately, to get starting values. 
Both are time-consuming, and to save time and trouble, we use the method described in [this paper](http://papers.ssrn.com/sol3/papers.cfm?abstract_id=420371), which is also described in [this paper](http://www.stata-journal.com/article.html?article=st0354). 

To estimate things using this method, you can read the latter paper above, and should download and install the `AMCMC` `Stata` module from `SSC`. It can be found [here](https://ideas.repec.org/c/boc/bocode/s457613.html).  

Since the price model is a little simpler and faster, we start by estimating it. The first thing we need to do is import some modules, find the current working directory, and then move over there within stata: 

In [3]:
import ipystata
import os

CWD = os.getcwd()
print(CWD)

C:\Users\mjbaker\Documents\GitHub\NewsHour


In [4]:
%%stata
clear all
cd C:\Users\mjbaker\Documents\GitHub\NewsHour


C:\Users\mjbaker\Documents\GitHub\NewsHour


Get our data (which we assembled in Part I) - it is an averaged data set that should be ready to use:

In [6]:
%%stata
use "\Data\AveragedDataDyno.dta", clear


file \Data\AveragedDataDyno.dta not found
r(601);


Oops first have to make the dynamic data set!